In [0]:
import numpy as np         # linear algebra
import sklearn as sk       # machine learning
import pandas as pd        # reading in data files, data cleaning
import matplotlib.pyplot as plt   # for plotting
import seaborn as sns      # visualization tool

file_id = '172V7yK-DjsxoaejDlOEl9S1R4A_GdP7U'
link = 'https://drive.google.com/uc?export=download&id={FILE_ID}'
csv_url = link.format(FILE_ID = file_id)

data = pd.read_csv(csv_url)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
Gender    500 non-null object
Height    500 non-null int64
Weight    500 non-null int64
Index     500 non-null int64
dtypes: int64(3), object(1)
memory usage: 15.7+ KB


In [0]:
data.head()

,Gender,Height,Weight,Index
0,Male,174,96,4
1,Male,189,87,2
2,Female,185,110,4
3,Female,195,104,3
4,Male,149,61,3


        0 - extremely weak
        1 - weak
        2 - normal
        3 - overweight
        4 - obese
        5 - extremely obese

In [0]:
data['Sex'] = data['Gender']
data['Sex'].replace(['Female', 'Male'], [0, 1], inplace=True)

data.head()

,Gender,Height,Weight,Index,Sex
0,Male,174,96,4,1
1,Male,189,87,2,1
2,Female,185,110,4,0
3,Female,195,104,3,0
4,Male,149,61,3,1


In [0]:
height_m = 0.01 * data['Height'] 
data['BMI'] = data['Weight'] / height_m**2

data.head()

,Gender,Height,Weight,Index,Sex,BMI
0,Male,174,96,4,1,31.708284
1,Male,189,87,2,1,24.355421
2,Female,185,110,4,0,32.140248
3,Female,195,104,3,0,27.350427
4,Male,149,61,3,1,27.476240


In [0]:
def z_trans(data):
  return (data - data.mean()) / data.std()

data['H'] = z_trans(data['Height'])
data['W'] = z_trans(data['Weight'])

data.head()

,Gender,Height,Weight,Index,Sex,BMI,H,W
0,Male,174,96,4,1,31.708284,0.247691,-0.308808
1,Male,189,87,2,1,24.355421,1.163707,-0.586735
2,Female,185,110,4,0,32.140248,0.919436,0.123523
3,Female,195,104,3,0,27.350427,1.530113,-0.061762
4,Male,149,61,3,1,27.476240,-1.279003,-1.389635


In [0]:
X = data[['Sex', 'H', 'W']]
y = data['Index']

In [0]:
data['zBMI'] = z_trans(data['BMI']);

data.head()

,Gender,Height,Weight,Index,Sex,BMI,H,W,zBMI
0,Male,174,96,4,1,31.708284,0.247691,-0.308808,-0.433708
1,Male,189,87,2,1,24.355421,1.163707,-0.586735,-0.960205
2,Female,185,110,4,0,32.140248,0.919436,0.123523,-0.402777
3,Female,195,104,3,0,27.350427,1.530113,-0.061762,-0.745750
4,Male,149,61,3,1,27.476240,-1.279003,-1.389635,-0.736741


In [0]:
from sklearn.neural_network import MLPClassifier
X_3 = data[['H', 'W', 'zBMI']]
y = data['Index']


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_3, y, test_size = 0.2, random_state=0)


clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(3, 3), random_state=1)
clf.fit(X_train, y_train)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(3, 3), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [0]:
y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

# [coef.shape for coef in clf.coefs_]
# clf.predict_proba(X_test)  

In [0]:
from sklearn.metrics import accuracy_score

print('Training set R^2 =', accuracy_score(y_train, y_pred_train))
print('Test set R^2 =', accuracy_score(y_test, y_pred_test))

Training set R^2 = 0.9375
Test set R^2 = 0.94


Some general rules about the hidden layer are the following based on this paper:* Approximating Number of Hidden layer neurons in Multiple Hidden Layer BPNN Architecture* by Saurabh Karsoliya.

In general:

*   The number of hidden layer neurons are 2/3 (or 70% to 90%) of the size of the input layer.
*   The number of hidden layer neurons should be less than twice of the number of neurons in input layer.
*   The size of the hidden layer neurons is between the input layer size and the output layer size.



In [0]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(max_iter=100)

parameter_space = {
    'hidden_layer_sizes': [(3,3,3), (5,2), (3,2), (5,5), (5,6), (2,3), (3,3), (4,4)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.0001, 0.05, 1e-5, 0.01, 0.001],
    'learning_rate': ['constant', 'adaptive', 'invscaling'],
}

from sklearn.model_selection import GridSearchCV

clf = GridSearchCV(mlp, parameter_space, n_jobs=-1, cv=3)
clf.fit(X_train, y_train)

# Best paramete set
print('Best parameters found:\n', clf.best_params_)

# All results
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))
    
y_true, y_pred = y_test , clf.predict(X_test)

from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best parameters found:
 {'activation': 'tanh', 'alpha': 0.05, 'hidden_layer_sizes': (4, 4), 'learning_rate': 'adaptive', 'solver': 'lbfgs'}
0.545 (+/-0.188) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'constant', 'solver': 'sgd'}
0.367 (+/-0.289) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'constant', 'solver': 'adam'}
0.943 (+/-0.019) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'constant', 'solver': 'lbfgs'}
0.580 (+/-0.117) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'adaptive', 'solver': 'sgd'}
0.463 (+/-0.263) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'adaptive', 'solver': 'adam'}
0.943 (+/-0.032) for {'activation': 'identity', 'alpha': 0.0001, 'hidden_layer_sizes': (3, 3, 3), 'learning_rate': 'adaptive', 'solver':

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
clf = MLPClassifier(activation='tanh', alpha=0.05, hidden_layer_sizes=(4, 4), learning_rate='adaptive', solver='lbfgs')
clf.fit(X_train, y_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

print('Training set R^2 =', accuracy_score(y_train, y_pred_train))
print('Test set R^2 =', accuracy_score(y_test, y_pred_test))

Training set R^2 = 0.9725
Test set R^2 = 0.97
